In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
# enable high-resolution figure
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
import matplotlib.pyplot as plt
from experiments.summarize import main as summarize_main
from pathlib import Path
import math

In [ ]:
RESULTS_DIR = Path("results/sweeps")
DATA = {}
KEYS = None
for method_dir in RESULTS_DIR.iterdir():
    method_name = str(method_dir).split("/")[-1]
    print(method_name)
    n_edit_folders = list(method_dir.glob("*_edits_setting_*"))
    for n_edit_folder in n_edit_folders:
        n_edits = int(str(n_edit_folder.name).split("/")[-1].split("_")[0])
        setting_id = str(n_edit_folder.name).split("/")[-1].split("_")[-1]
        try:
            res = summarize_main(n_edit_folder.relative_to("results"), ["run_000"])[0]

            DATA[method_name] = DATA.get(method_name, {})
            DATA[method_name][n_edits] = DATA[method_name].get(n_edits, {})
            DATA[method_name][n_edits][setting_id] = res

            if KEYS is None:
                KEYS = list(res.keys())
        except:
            pass

print({k: list(v.keys()) for k, v in DATA.items()})

In [ ]:
for method, all_n_edits in sorted([(k, v) for k, v in DATA.items()]):
    for n_edits, results in sorted([(k, v) for k, v in all_n_edits.items()]):
        w = 4
        h = math.ceil(len(KEYS) / w)
        plt.figure(figsize=(w * 3.5, h * 2.5))
        if "run_dir" in KEYS:
            KEYS.pop(KEYS.index("run_dir"))
        for i, key in enumerate(KEYS):
            plt.subplot(w, h, i + 1)

            try:
                setting_ids = list(map(int, results.keys()))
                values = [
                    f[0] if (type(f := results[str(n)][key]) is tuple) else f
                    for n in setting_ids
                ]
                plt.plot(setting_ids, values, marker="o", markersize=4, label=method)
                plt.xlabel("setting_id")
                plt.ylabel("metric value")
                plt.title(f"{n_edits} edits: {key}")
                plt.legend()
            except:
                pass

        plt.tight_layout()
        plt.show()